<a href="https://colab.research.google.com/github/MorganPeju/GridRival_Combo/blob/main/GridRival_combo_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''
Load libraries
'''
import pandas as pd
from itertools import combinations, product
from csv import reader

In [11]:
'''
Definition of functions
'''
def load_data(path):
  '''
  Function thats loads data

  Input:
    - path : path to data file
  Output:
    - tuples : list of tuples (pilote, value)
  '''
  df = pd.read_csv(path)
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1])
  tuples = list(df.itertuples(index=False, name=None))
  return tuples

def combination_pilote(pilote, n=2):
  '''
  Function that computes all combinations of n drivers
  Inputs:
    - pilote : list of K drivers
    - n : number of drivers in each combinations
  Output:
    - combo : list that contains each combination
  '''
  return list(combinations(pilote, n))

def add_team(pilote,ecurie):
  '''
  Function that add teams for each combination

  Inputs: 
    - pilote : list of drivers
    - ecurie : list of teams
  Output:
    - combo : list that contains each combination of drivers + 1 team
  '''
  combo = []
  for i in pilote:
      for j in ecurie:
          cat = list(i)
          cat.append(j)
          combo.append(cat)
  return combo

def build_df(list_of_tuples, col=['X1', 'X2']):
  '''
  Function that builds a dataframe from a list of tuples

  Inputs: 
    - list_of_tuples : list of n tuples
    - col : columns' names of the dataframe
  Output:
    - dataframe of two columns correponding to the first and second value of each tuple
  '''
  L_f = []
  for comb in list_of_tuples:
      L = [x[0] for x in comb]
      L_sum = sum(j for i, j in comb)
      L = "+".join(L)
      L_f.append((L,L_sum))
  
  return pd.DataFrame(L_f, columns=col)

def filter_df(dataframe, sort_by_index=0, ascending=False, min=0, max=200):
    '''
    Function that sorts, filters and splits the dataframe 
    - Sort by value's column
    - Filter on value : between a min and max
    - Split into the following structure:
    ||Pilote#1|Pilote#2|Pilote#3|Pilote#4|Pilote#4|Team|Value||

    Inputs:
      - dataframe : contains all combos and their value
      - sort_by_index : correponds to "Value"'s index column
      - ascending : Boolean if we want an ascending and descending sort
      - min : minimum value to filter the dataframe (value's column)
      - max : maximum value to filter the dataframe (value's column)
    Output:
      - dataframe sorted, filtered and split : [n rows * 7 cols]
    '''
    # Sort
    sort_by = dataframe.columns[sort_by_index]
    dataframe = dataframe.sort_values(sort_by, ascending=ascending)
    # Filter
    dataframe = dataframe.loc[(dataframe[sort_by] >= min) & (dataframe[sort_by] <= max)]      
    # Split
    df2 = dataframe.iloc[:, 0].str.split('+',expand=True)
    dataframe = pd.concat([df2,dataframe.iloc[:, 1]], axis=1) 
  
    # Noms colonnes du datadrame
    for i in range(0,6):
        if i==5 :
            dataframe.columns.values[i] = "Team"
        elif i==6 :
            dataframe.columns.values[i] = "Valeur"
        else:
            dataframe.columns.values[i] = "Pilote#" +str(i+1)
    return dataframe.reset_index(drop=True)

def filter_df_by_pilote_team(dataframe, pil1=None, pil2=None,pil3=None,pil4=None,pil5=None, team=None):
  '''
  Function that applies a filter on the dataframe to only keep combos with specific drivers/team
  
  Inputs:
    - dataframe [n rows * 7 cols] with this structure ||Pilote#1|Pilote#2|Pilote#3|Pilote#4|Pilote#4|Team|Value||
    - pil1 to pil5 : a string of the driver's acronym. (i.e Charles Leclerc --> "LEC")
    - team : a string containing the name of a F1 team

  Output:
    - Filtered dataframe : [n' rows * 7 cols]
  '''
  combo = [pil1,pil2,pil3,pil4,pil5,team]
  cols = dataframe.columns[0:6]
  for ele in combo:
      if ele is not None:
          dataframe = dataframe[(dataframe[cols]==ele).any(axis=1)]
  return dataframe.reset_index(drop=True)


def main(budget_min, budget_max, pil1=None, pil2=None,pil3=None,pil4=None,pil5=None, team=None):
  '''
  Main function
  '''
  url_drivers = 'https://raw.githubusercontent.com/MorganPeju/GridRival_Combo/main/data/drivers.csv'
  url_teams = 'https://raw.githubusercontent.com/MorganPeju/GridRival_Combo/main/data/teams.csv'
  pilote = load_data(url_drivers)
  ecurie = load_data(url_teams)

  BUDGET_MIN = budget_min
  BUDGET_MAX = budget_max

  combinaisons = combination_pilote(pilote, 5)

  combo = add_team(combinaisons,ecurie)

  df = build_df(combo,['Combo', 'Valeur'])

  df = filter_df(df, 1, False, min=BUDGET_MIN, max=BUDGET_MAX)

  df = filter_df_by_pilote_team(df, pil1, pil2, pil3, pil4, pil5,team)
  
  print(df)

In [18]:
'''
Define min and max budget of your GridRival team
Define if you want specific drivers or team

Drivers' examples:
"VER" : Max Verstappen
"HAM" : Lewis Hamilton
"LEC" : Charles Leclerc
"RIC" : Daniel Ricciardo
"GAS" : Pierre Gasly

Teams' examples: 
"Mercedes", "Ferrari","Red Bull", "McLaren", "AlphaTauri"
"Alfa Romeo", "Aston Martin", "Haas", "Williams", "Alpine"

'''

### Your budget
bud_max = 105.1
bud_min= bud_max

### Your team
# Replace Nones by acronyms of the drivers you want, or keep None
driver1, driver2, driver3, driver4, driver5 = "LEC", "BOT", None, None, None
scuderia = "Ferrari" # Replace by a team name or None

In [19]:
'''
Run this code to get your combos
'''
# pd.set_option('max_rows', 99999) # Uncomment this line to see every rows
main(budget_min=round(bud_min,3), budget_max=round(bud_max,3), pil1=driver1, pil2=driver2, pil3=driver3, pil4=driver4, pil5=driver4, team=scuderia)

  Pilote#1 Pilote#2 Pilote#3 Pilote#4 Pilote#5     Team  Valeur
0      HAM      LEC      BOT      ZHO      LAT  Ferrari   105.1
